Домашнее задание (часть 2)
1. Придумать новые признаки (у нас их всего 5 было) - сделать дополнительный feature engineering
2. Для подходов с одной моделью поварьировать параметры CatBoostClassifier либо даже попробовать другой алгоритм (опционально уже). 
3. Для пункта 2 (подходов на основе одной модели) вывести значимость признаков, основываясь на model.estimator.feature_importances_ и model.estimator.feature_names_. 
4. Дополнить models_results дополнительными метриками (roc auc, logloss, f1). В конце вывести сводную таблицу всех показателей для всех подходов (как на основе)


In [1]:
%matplotlib inline

import pandas as pd
#import pandas as pd; pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split


# Чтение данных
df_clients = pd.read_csv('./data/clients.csv', index_col='client_id')
df_train = pd.read_csv('./data/uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('./data/uplift_test.csv', index_col='client_id')

# Извлечение признаков
df_features = df_clients.copy()
df_features['first_issue_time'] = \
    (pd.to_datetime(df_features['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_features['first_redeem_time'] = \
    (pd.to_datetime(df_features['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_features['issue_redeem_delay'] = df_features['first_redeem_time'] \
    - df_features['first_issue_time']
df_features = df_features.drop(['first_issue_date', 'first_redeem_date'], axis=1)

indices_train = df_train.index
indices_test = df_test.index
indices_learn, indices_valid = train_test_split(df_train.index, 
                                                test_size=0.4, 
                                                random_state=12)

In [2]:
#признаки
df_features.head(10)

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay
client_id,,,,,
000012768d,45,U,1501947648,1.515094e+09,13146559.0
000036f903,72,F,1491832463,1.492951e+09,1118613.0
000048b7a6,68,F,1544880791,NaN,NaN
000073194a,60,F,1495544174,1.511522e+09,15978107.0
00007c7133,67,U,1495469828,1.546277e+09,50806825.0
00007f9014,45,F,1503408901,1.550258e+09,46849457.0
0000a90cf7,45,U,1495283690,1.499518e+09,4234412.0
0000b59cec,46,U,1497002660,1.514400e+09,17397098.0
0000bb4e4e,36,M,1494345727,1.503681e+09,9334854.0


In [3]:
df_train.head(10)

,treatment_flg,target
client_id,,
000012768d,0,1
000036f903,1,1
00010925a5,1,1
0001f552b0,1,1
00020e7b18,1,1
000220a0a7,0,1
00022fd34f,1,1
0002ce2217,0,1
00031cbbe6,0,1


In [4]:
df_test.head(10)

""
client_id
000048b7a6
000073194a
00007c7133
00007f9014
0000a90cf7
0000b59cec
0000bb4e4e
0000bcec9c
0000eecb82


**Поле treatment_flg - было воздействие или нет.**

In [5]:
df_train.groupby('treatment_flg')['target'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment_flg,,,,,,,,
0,100058.0,0.603280,0.489219,0.0,0.0,1.0,1.0,1.0
1,99981.0,0.636511,0.481006,0.0,0.0,1.0,1.0,1.0


In [6]:
df_train['target'].value_counts()

1    124002
0     76037
Name: target, dtype: int64

In [7]:
df_features.describe()

,age,first_issue_time,first_redeem_time,issue_redeem_delay
count,400162.000000,4.001620e+05,3.646930e+05,3.646930e+05
mean,46.488112,1.515519e+09,1.530007e+09,1.558871e+07
std,43.871218,1.759150e+07,1.888414e+07,1.351122e+07
min,-7491.000000,1.491330e+09,1.491904e+09,-3.701689e+06
25%,34.000000,1.500667e+09,1.514128e+09,5.368447e+06
50%,45.000000,1.510845e+09,1.528319e+09,1.148628e+07
75%,59.000000,1.529518e+09,1.546628e+09,2.185546e+07
max,1901.000000,1.552687e+09,1.574212e+09,8.198520e+07


In [8]:
print(f"age>100,count={df_features[df_features['age']>100]['age'].count()} from {len(df_features)} or {(df_features[df_features['age']>100]['age'].count())/(len(df_features))*100}%")


age>100,count=1049 from 400162 or 0.262143831748142%


**Добавим новый признак- age_group**

In [9]:
#age to categorical column
def age_lab(data) :
    
    if data["age"] <= 20 :
        return "age_17-20"
    elif (data["age"] > 20) & (data["age"] <= 25 ):
        return "age_20-25"
    elif (data["age"] > 25) & (data["age"] <= 30) :
        return "age_25-30"
    elif (data["age"] > 30) & (data["age"] <= 40) :
        return "age_30-40"
    elif (data["age"] > 40) & (data["age"] <= 50) :
        return "age_40-50"
    elif (data["age"] > 50) & (data["age"] <= 60) :
        return "age_50-60"
    elif (data["age"] > 60) & (data["age"] <= 70) :
        return "age_60-70"
    elif data["age"] > 70 :
        return "age_gt_70"
    
df_features["age_group"] = df_features.apply(lambda df_features:age_lab(df_features), axis = 1)
    


In [10]:
df_features.head(10)

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,age_group
client_id,,,,,,
000012768d,45,U,1501947648,1.515094e+09,13146559.0,age_40-50
000036f903,72,F,1491832463,1.492951e+09,1118613.0,age_gt_70
000048b7a6,68,F,1544880791,NaN,NaN,age_60-70
000073194a,60,F,1495544174,1.511522e+09,15978107.0,age_50-60
00007c7133,67,U,1495469828,1.546277e+09,50806825.0,age_60-70
00007f9014,45,F,1503408901,1.550258e+09,46849457.0,age_40-50
0000a90cf7,45,U,1495283690,1.499518e+09,4234412.0,age_40-50
0000b59cec,46,U,1497002660,1.514400e+09,17397098.0,age_40-50
0000bb4e4e,36,M,1494345727,1.503681e+09,9334854.0,age_30-40


In [11]:
df_features = pd.get_dummies(data = df_features,columns = ["age_group"] )

In [12]:
df_features.head(10)

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,age_group_age_17-20,age_group_age_20-25,age_group_age_25-30,age_group_age_30-40,age_group_age_40-50,age_group_age_50-60,age_group_age_60-70,age_group_age_gt_70
client_id,,,,,,,,,,,,,
000012768d,45,U,1501947648,1.515094e+09,13146559.0,0,0,0,0,1,0,0,0
000036f903,72,F,1491832463,1.492951e+09,1118613.0,0,0,0,0,0,0,0,1
000048b7a6,68,F,1544880791,NaN,NaN,0,0,0,0,0,0,1,0
000073194a,60,F,1495544174,1.511522e+09,15978107.0,0,0,0,0,0,1,0,0
00007c7133,67,U,1495469828,1.546277e+09,50806825.0,0,0,0,0,0,0,1,0
00007f9014,45,F,1503408901,1.550258e+09,46849457.0,0,0,0,0,1,0,0,0
0000a90cf7,45,U,1495283690,1.499518e+09,4234412.0,0,0,0,0,1,0,0,0
0000b59cec,46,U,1497002660,1.514400e+09,17397098.0,0,0,0,0,1,0,0,0
0000bb4e4e,36,M,1494345727,1.503681e+09,9334854.0,0,0,0,1,0,0,0,0


In [13]:
#Для удобства объявим некоторые переменные:
X_train = df_features.loc[indices_learn, :]
y_train = df_train.loc[indices_learn, 'target']
treat_train = df_train.loc[indices_learn, 'treatment_flg']

X_val = df_features.loc[indices_valid, :]
y_val = df_train.loc[indices_valid, 'target']
treat_val =  df_train.loc[indices_valid, 'treatment_flg']

X_train_full = df_features.loc[indices_train, :]
y_train_full = df_train.loc[:, 'target']
treat_train_full = df_train.loc[:, 'treatment_flg']

X_test = df_features.loc[indices_test, :]

cat_features = ['gender']

models_results = {
    'approach': [],
    'uplift@30%': [],
    'uplift_auc_score':[],
    'weighted_average_uplift':[],
    'iterations,learning_rate,depth':[]
}

In [14]:
!pip install scikit-uplift

In [15]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

In [16]:
iterations_arr=[15,20,30]
learning_rate_arr=[0.01,0.05,0.1]
depth_arr=[3,5,7]

### Используем подход с 1 моделью - 1. Одна модель с признаком коммуникации

In [17]:

from sklift.metrics import uplift_at_k,uplift_auc_score,weighted_average_uplift,response_rate_by_percentile
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel

# sklift поддерживает любые модели, 
# которые удовлетворяют соглашениями scikit-learn
# Для примера воспользуемся catboost
from catboost import CatBoostClassifier

def explore_solo_model(iterations=20,learning_rate=0.01,depth=5):    
    sm = SoloModel(CatBoostClassifier(iterations=iterations,learning_rate=learning_rate,depth=depth, thread_count=2, random_state=42, silent=True))
    sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

    uplift_sm = sm.predict(X_val)

    sm_score = uplift_at_k(y_true=y_val, uplift=uplift_sm, treatment=treat_val, strategy='by_group', k=0.3)

    models_results['approach'].append('SoloModel')
    models_results['uplift@30%'].append(sm_score)
    models_results['uplift_auc_score'].append(uplift_auc_score(y_true=y_val, uplift=uplift_sm, treatment=treat_val))
    models_results['weighted_average_uplift'].append(weighted_average_uplift(y_true=y_val, uplift=uplift_sm, treatment=treat_val))
    models_results['iterations,learning_rate,depth'].append(str(iterations)+"; "+str(learning_rate)+"; "+str(depth))

    # Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
    sm_trmnt_preds = sm.trmnt_preds_
    # И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
    sm_ctrl_preds = sm.ctrl_preds_

    # Отрисуем распределения вероятностей и их разность (uplift)
    #plot_uplift_preds(trmnt_preds=sm_trmnt_preds, ctrl_preds=sm_ctrl_preds);
    
    print(pd.DataFrame({
        'feature_name': sm.estimator.feature_names_,
        'feature_score': sm.estimator.feature_importances_
    }).sort_values('feature_score', ascending=False).reset_index(drop=True))


for iterations in iterations_arr:
    for learning_rate in learning_rate_arr:
        for depth in depth_arr:
            explore_solo_model(iterations,learning_rate,depth)

           feature_name  feature_score
0     first_redeem_time      55.533697
1    issue_redeem_delay      39.439146
2      first_issue_time       2.302295
3                   age       1.372662
4             treatment       1.325305
5   age_group_age_20-25       0.026895
6                gender       0.000000
7   age_group_age_17-20       0.000000
8   age_group_age_25-30       0.000000
9   age_group_age_30-40       0.000000
10  age_group_age_40-50       0.000000
11  age_group_age_50-60       0.000000
12  age_group_age_60-70       0.000000
13  age_group_age_gt_70       0.000000
           feature_name  feature_score
0     first_redeem_time      67.015085
1    issue_redeem_delay      24.583325
2      first_issue_time       4.313457
3                   age       2.302716
4             treatment       1.555172
5   age_group_age_60-70       0.230245
6                gender       0.000000
7   age_group_age_17-20       0.000000
8   age_group_age_20-25       0.000000
9   age_group_age_25-30  

           feature_name  feature_score
0     first_redeem_time      47.770581
1    issue_redeem_delay      41.680483
2                   age       5.132024
3      first_issue_time       3.051818
4             treatment       1.838660
5   age_group_age_gt_70       0.487978
6   age_group_age_20-25       0.038455
7                gender       0.000000
8   age_group_age_17-20       0.000000
9   age_group_age_25-30       0.000000
10  age_group_age_30-40       0.000000
11  age_group_age_40-50       0.000000
12  age_group_age_50-60       0.000000
13  age_group_age_60-70       0.000000
           feature_name  feature_score
0     first_redeem_time      63.559708
1    issue_redeem_delay      20.916838
2      first_issue_time       5.950395
3                   age       5.481998
4             treatment       2.949590
5   age_group_age_gt_70       0.757538
6   age_group_age_60-70       0.240022
7   age_group_age_20-25       0.143912
8                gender       0.000000
9   age_group_age_17-20  

### Используем подход с 1 моделью - 2. Трансформация классов

In [19]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds

# sklift поддерживает любые модели, 
# которые удовлетворяют соглашениями scikit-learn
# Для примера воспользуемся catboost
from catboost import CatBoostClassifier
from sklift.models import ClassTransformation

def explore_classtransformation_model(iterations=20,learning_rate=0.01,depth=5):   
    ct = ClassTransformation(CatBoostClassifier(iterations=iterations,learning_rate=learning_rate,depth=depth, thread_count=2, random_state=42, silent=True))
    ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

    uplift_ct = ct.predict(X_val)

    ct_score = uplift_at_k(y_true=y_val, uplift=uplift_ct, treatment=treat_val, strategy='by_group', k=0.3)

    models_results['approach'].append('ClassTransformation')
    models_results['uplift@30%'].append(ct_score)       
    models_results['uplift_auc_score'].append(uplift_auc_score(y_true=y_val, uplift=uplift_ct, treatment=treat_val))
    models_results['weighted_average_uplift'].append(weighted_average_uplift(y_true=y_val, uplift=uplift_ct, treatment=treat_val))
    models_results['iterations,learning_rate,depth'].append(str(iterations)+"; "+str(learning_rate)+"; "+str(depth))

    # Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
    #ct_trmnt_preds = ct.trmnt_preds_
    # И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
    #ct_ctrl_preds = ct.ctrl_preds_

    # Отрисуем распределения вероятностей и их разность (uplift)

    #plot_uplift_preds(trmnt_preds=ct.trmnt_preds_, ctrl_preds=ct.ctrl_preds_);
    
    print(pd.DataFrame({
        'feature_name': ct.estimator.feature_names_,
        'feature_score': ct.estimator.feature_importances_
    }).sort_values('feature_score', ascending=False).reset_index(drop=True))
    
for iterations in iterations_arr:
    for learning_rate in learning_rate_arr:
        for depth in depth_arr:
            explore_classtransformation_model(iterations,learning_rate,depth)

C:\Users\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  if sys.path[0] == '':


           feature_name  feature_score
0     first_redeem_time      97.650019
1                   age       0.986823
2    issue_redeem_delay       0.633375
3      first_issue_time       0.591271
4   age_group_age_60-70       0.109561
5   age_group_age_gt_70       0.028950
6                gender       0.000000
7   age_group_age_17-20       0.000000
8   age_group_age_20-25       0.000000
9   age_group_age_25-30       0.000000
10  age_group_age_30-40       0.000000
11  age_group_age_40-50       0.000000
12  age_group_age_50-60       0.000000


C:\Users\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  if sys.path[0] == '':


           feature_name  feature_score
0     first_redeem_time      96.605819
1    issue_redeem_delay       1.071119
2      first_issue_time       1.023400
3                   age       0.890860
4   age_group_age_60-70       0.150490
5   age_group_age_20-25       0.114632
6   age_group_age_30-40       0.094432
7   age_group_age_gt_70       0.049249
8                gender       0.000000
9   age_group_age_17-20       0.000000
10  age_group_age_25-30       0.000000
11  age_group_age_40-50       0.000000
12  age_group_age_50-60       0.000000


C:\Users\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  if sys.path[0] == '':


...

           feature_name  feature_score
0     first_redeem_time      96.225867
1                   age       1.237656
2    issue_redeem_delay       1.140143
3      first_issue_time       0.775574
4   age_group_age_60-70       0.415952
5   age_group_age_20-25       0.078321
6   age_group_age_gt_70       0.055743
7                gender       0.033243
8   age_group_age_40-50       0.022198
9   age_group_age_25-30       0.015304
10  age_group_age_17-20       0.000000
11  age_group_age_30-40       0.000000
12  age_group_age_50-60       0.000000


C:\Users\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  if sys.path[0] == '':


           feature_name  feature_score
0     first_redeem_time      87.822312
1                   age       3.522534
2      first_issue_time       3.428512
3    issue_redeem_delay       2.192845
4                gender       1.826287
5   age_group_age_60-70       0.497503
6   age_group_age_30-40       0.284071
7   age_group_age_gt_70       0.160124
8   age_group_age_20-25       0.101589
9   age_group_age_50-60       0.101277
10  age_group_age_40-50       0.062946
11  age_group_age_17-20       0.000000
12  age_group_age_25-30       0.000000


C:\Users\Windows\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  if sys.path[0] == '':


           feature_name  feature_score
0     first_redeem_time      77.273149
1      first_issue_time       5.960329
2    issue_redeem_delay       4.908350
3                   age       4.133596
4                gender       2.688843
5   age_group_age_30-40       1.429959
6   age_group_age_20-25       1.056998
7   age_group_age_60-70       0.797080
8   age_group_age_gt_70       0.537939
9   age_group_age_25-30       0.420628
10  age_group_age_50-60       0.320669
11  age_group_age_40-50       0.239037
12  age_group_age_17-20       0.233423


In [21]:
pd.DataFrame(data=models_results).sort_values('uplift@30%', ascending=False)

,approach,uplift@30%,uplift_auc_score,weighted_average_uplift,"iterations,learning_rate,depth"
37,ClassTransformation,0.069365,0.028001,0.026001,20; 0.01; 5
13,SoloModel,0.069098,0.026482,0.024911,20; 0.05; 5
10,SoloModel,0.068942,0.026952,0.024623,20; 0.01; 5
7,SoloModel,0.068628,0.024071,0.024637,15; 0.1; 5
53,ClassTransformation,0.068531,0.026056,0.026276,30; 0.1; 7
31,ClassTransformation,0.068501,0.026249,0.025764,15; 0.05; 5
50,ClassTransformation,0.067652,0.025059,0.025809,30; 0.05; 7
46,ClassTransformation,0.067583,0.028713,0.026161,30; 0.01; 5
23,SoloModel,0.067391,0.025340,0.024523,30; 0.05; 7
32,ClassTransformation,0.067257,0.027493,0.026615,15; 0.05; 7


**Вывод - лучшая модель: ClassTransformation, где uplift@30%=0.069365; iterations=20; learning_rate=0.01; depth=5**